In [13]:
import pandas as pd
# from pandarallel import pandarallel
import numpy as np
import seaborn as sns
import plotly.express as px
from tqdm import tqdm

import warnings

warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize': (20, 10), 'figure.facecolor': 'white'})
sns.set_palette("viridis")
# pandarallel.initialize(progress_bar=True)

# Stage 1 - Get Data

In [5]:
train = (
    pd.read_csv('../data/raw/additional_data/building_20230808.csv')
    .merge(
        pd.read_csv('../data/raw/additional_data/district_20230808.csv'),
        left_on='district_id',
        right_on='id',
        how='left',
        suffixes=('_building', '_district')
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/prefix_20230808.csv'),
        left_on='prefix_id',
        right_on='id',
        how='left',
        suffixes=('_district', '_prefix')
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/town_20230808.csv'),
        left_on='town_id',
        right_on='id',
        how='left',
        suffixes=('_prefix', '_town')
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/geonim_20230808.csv'),
        left_on='geonim_id',
        right_on='id',
        how='left',
        suffixes=('_town', '_geonim')
    )
    .rename(columns={'short_name': 'short_name_geonim', 'only_name': 'only_name_geonim'})
    # [['id_building', 'full_address', 'short_address', 'house', 'corpus', 'liter', 'name_town',
    #   'short_name_town', 'name_geonim', 'short_name_geonim', 'only_name_geonim']]  # 'id_building', 'post_prefix'
)
train

,id_building,prefix_id,district_id,house,corpus,liter,villa,parcel,full_address,is_updated_building,...,is_updated_town,is_actual_town,has_buildings_town,id,type_id,name_geonim,short_name_geonim,is_updated,is_actual,only_name_geonim
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,...,True,True,True,4801.0,12.0,Кедринская улица,Кедринская ул.,True,True,Кедринская
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,...,True,True,True,3.0,12.0,Пляжевая улица,Пляжевая ул.,True,True,Пляжевая
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, до...",True,...,True,True,True,2194.0,10.0,проспект Маршала Жукова,пр. Маршала Жукова,True,True,Маршала Жукова
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, д...",True,...,True,True,True,319.0,12.0,Центральная улица,Центральная ул.,True,True,Центральная
4,185368,4224,38,28,NaN,Б,NaN,NaN,"поселок Песочный, Речная улица, дом 28, литера Б",True,...,True,True,True,477.0,12.0,Речная улица,Речная ул.,True,True,Речная
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166640,18941,5289,32,3,NaN,В,NaN,NaN,"г.Санкт-Петербург, Бородинская улица, дом 3, л...",True,...,True,True,True,1748.0,12.0,Бородинская улица,Бородинская ул.,True,True,Бородинская
166641,18942,5031,35,34/3,NaN,А,NaN,NaN,"посёлок Александровская, 2-я линия, дом 34/3, ...",True,...,True,True,True,2861.0,6.0,2-я линия,2-я линия,True,True,2-я
166642,18943,5623,38,39,NaN,NaN,NaN,NaN,"посёлок Ушково, Детская улица, дом 39",True,...,True,True,True,4.0,12.0,Детская улица,Детская ул.,True,True,Детская
166643,18944,5787,34,25,NaN,Б,NaN,NaN,"г.Санкт-Петербург, Заусадебная улица, дом 25, ...",True,...,True,True,True,1316.0,12.0,Заусадебная улица,Заусадебная ул.,True,True,Заусадебная


# Step 2 - Get Embeddings

# Step 3 - Model

In [6]:
QUERY = ["г.Санкт-Петербург, дом 223-225, наб. Обводного канала"]

# add logic to topn selection

In [7]:
from sentence_transformers import SentenceTransformer, util
import pickle

In [9]:
import pickle

with open('embeddings_list.pkl', 'rb') as f:
    embeddings_list = pickle.load(f)

train_emb_only = pd.read_pickle("names_embeddings_list.pkl")

In [10]:
from sentence_transformers import SentenceTransformer, util
import pickle

def get_best_matches(query, top_n, embeddings: np.array, sentences):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = embeddings.dot(query_embedding.T).ravel()
    best = np.argpartition(scores, -top_n)[-top_n:]
    sentences['scores'] = scores
    sentences.loc[list(best), 'lev_distance'] = sentences.loc[list(best), 'full_address'].apply(lambda x: distance(query, x))
    return sentences[['id_building', 'full_address', 'scores', 'lev_distance']].loc[list(best)].sort_values(by='lev_distance').reset_index(drop=True)

In [11]:
with open("../notebooks/embeddings_list.pkl", "rb") as f:
    embeddings_list = pickle.load(f)

In [14]:
from Levenshtein import distance
get_best_matches(QUERY, 5, np.array(embeddings_list), train_emb_only)

,id_building,full_address,scores,lev_distance
0,9534,"г.Санкт-Петербург, набережная Обводного канала...",0.973808,59.0
1,33462,"г.Санкт-Петербург, набережная Обводного канала...",0.966640,69.0
2,40214,"г.Санкт-Петербург, набережная Обводного канала...",0.966658,69.0
3,56448,"г.Санкт-Петербург, набережная Обводного канала...",0.966934,69.0
4,187158,"г.Санкт-Петербург, набережная Обводного канала...",0.966934,69.0


In [15]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [16]:
embeddings = np.array(embeddings_list)
sentences = train_emb_only['full_address']

In [17]:
query_embedding= model.encode(QUERY, convert_to_tensor=True)

In [18]:
topn = 5
scores = embeddings.dot(query_embedding.T).ravel()
best = np.argpartition(scores, -topn)[-topn:]

In [19]:
list(sentences.loc[list(best)])

['г.Санкт-Петербург, набережная Обводного канала, дом 223-225, литера К',
 'г.Санкт-Петербург, набережная Обводного канала, дом 223-225, литера О',
 'г.Санкт-Петербург, набережная Обводного канала, дом 223-225',
 'г.Санкт-Петербург, набережная Обводного канала, дом 223-225, литера П',
 'г.Санкт-Петербург, набережная Обводного канала, дом 223-225, литера П']

# add stage with levenstein distance

In [70]:
from Levenshtein import distance
sentences = pd.DataFrame(sentences)
# sentences.loc['lev_distance'] = 0
for indx, embd in zip(list(best), sentences.loc[list(best), 'full_address']):
    sentences.loc[indx, 'lev_distance'] = distance(QUERY, embd)

In [83]:
sentences.loc[list(best), 'lev_distance'] = sentences.loc[list(best), 'full_address'].apply(lambda x: distance(QUERY, x))

In [79]:
sentences.loc[list(best), 'lev_distance']

37720    69.0
44194    69.0
2929     59.0
59784    69.0
7        69.0
Name: lev_distance, dtype: float64

In [74]:
distance(QUERY, 'город Пушкин, Кедринская улица, дом 12')

38

In [71]:
sentences.loc[list(best), 'full_address']

37720    г.Санкт-Петербург, набережная Обводного канала...
44194    г.Санкт-Петербург, набережная Обводного канала...
2929     г.Санкт-Петербург, набережная Обводного канала...
59784    г.Санкт-Петербург, набережная Обводного канала...
7        г.Санкт-Петербург, набережная Обводного канала...
Name: full_address, dtype: object

In [76]:
sentences.loc[list(best)].sort_values(by='lev_distance')

,full_address,lev_distance,1
2929,"г.Санкт-Петербург, набережная Обводного канала...",59.0,59.0
37720,"г.Санкт-Петербург, набережная Обводного канала...",69.0,69.0
44194,"г.Санкт-Петербург, набережная Обводного канала...",69.0,69.0
59784,"г.Санкт-Петербург, набережная Обводного канала...",69.0,69.0
7,"г.Санкт-Петербург, набережная Обводного канала...",69.0,69.0


test:
г.Санкт-Петербург, дом 223-225, наб. Обводного канала
г.Санкт-Петербург, дом 223-225
посёлок Солнечное, Комсомольская улица